In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from google.colab import drive as google


In [ ]:
google.mount("/content/Drive")

In [ ]:

# Attention Gate
def attention_block(x, g, inter_channels):
    theta_x = layers.Conv2D(inter_channels, 1, strides=1, padding='same')(x)
    phi_g = layers.Conv2D(inter_channels, 1, strides=1, padding='same')(g)
    add = layers.Add()([theta_x, phi_g])
    act = layers.Activation('relu')(add)
    psi = layers.Conv2D(1, 1, strides=1, padding='same', activation='sigmoid')(act)
    return layers.Multiply()([x, psi])

# U-Net with Attention for Classification
def unet_attention(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(pool2)
    conv3 = layers.Conv2D(256, 3, activation='relu', padding='same')(conv3)
    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    # Bottleneck
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(pool3)
    conv4 = layers.Conv2D(512, 3, activation='relu', padding='same')(conv4)

    # Decoder with Attention Blocks
    up3 = layers.Conv2DTranspose(256, kernel_size=2, strides=2, padding="same")(conv4)
    att3 = attention_block(conv3, up3, 256)
    merge3 = layers.Concatenate()([att3, up3])
    conv5 = layers.Conv2D(256, 3, activation="relu", padding="same")(merge3)

    up2 = layers.Conv2DTranspose(128, kernel_size=2, strides=2, padding="same")(conv5)
    att2 = attention_block(conv2, up2, 128)
    merge2 = layers.Concatenate()([att2, up2])
    conv6 = layers.Conv2D(128, 3, activation="relu", padding="same")(merge2)

    up1 = layers.Conv2DTranspose(64, kernel_size=2, strides=2, padding="same")(conv6)
    att1 = attention_block(conv1, up1, 64)
    merge1 = layers.Concatenate()([att1, up1])
    conv7 = layers.Conv2D(64, 3, activation="relu", padding="same")(merge1)


    # Classification Head
    gap = layers.GlobalAveragePooling2D()(conv7)
    output = layers.Dense(num_classes, activation='softmax')(gap)

    model = keras.Model(inputs, output)
    return model


In [ ]:
# Define model
input_shape = (256, 256, 3)  # Adjust size based on your MRI dataset
num_classes = 4  # (3 tumor types + 1 no tumor)
model = unet_attention(input_shape, num_classes)

# Compile model
model.compile(optimizer='adam',
               loss='categorical_crossentropy',
               metrics=['accuracy'])

# Data loading and augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

In [ ]:
import zipfile
import os

def unzip_file(zip_path, extract_to='.'):
    """
    Unzips a file to the specified directory.

    Parameters:
    zip_path (str): Path to the zip file.
    extract_to (str): Directory where the files should be extracted. Default is the current directory.
    """
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_to)
        print(f"Extracted files to: {os.path.abspath(extract_to)}")
    except Exception as e:
        print(f"Error: {e}")

unzip_file('/content/Drive/MyDrive/brain tumor.zip', '/content/braintumor_folder')


In [ ]:
IMG_SIZE = 256
TRAIN_DIR = "/content/braintumor_folder/Training"
TEST_DIR = "/content/braintumor_folder/Testing"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


training_dataset = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size = (IMG_SIZE,IMG_SIZE),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True
)
test_datagen = ImageDataGenerator(rescale=1./255)

test_dataset = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size = (IMG_SIZE,IMG_SIZE),
    batch_size = 32,
    class_mode = "categorical",
    shuffle = True
)

training_dataset.class_indices



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get a batch of images and labels
images, labels = next(training_dataset)  # Get one batch (32 images in this case)

# Get class names from class indices
class_labels = {v: k for k, v in training_dataset.class_indices.items()}  # Reverse the dictionary

plt.figure(figsize=(10, 10))
for i in range(9):  # Show 9 images
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])  # Images are already rescaled
    label_index = np.argmax(labels[i])  # Get the class index
    plt.title(class_labels[label_index])  # Show class name
    plt.axis("off")

plt.show()


In [ ]:
from tensorflow.keras.callbacks import Callback
class StopOnAccuracy(Callback):
  def __init__(self,target_acc=0.98):
    super(StopOnAccuracy, self).__init__()
    self.target_acc = target_acc

  def on_epoch_end(self, epoch, logs=None):
    logs = logs or {}
    if logs.get('accuracy')>=self.target_acc:
      print(f"\nReached {self.target_acc} , so the training is being stopped\n")
      self.model.stop_training = True

stop_on_accuracy = StopOnAccuracy(target_acc=0.96)

from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience= 15, monitor="val_loss", restore_best_weights = True, verbose = 1)


In [ ]:
print(model.to_json())  # Useful for saving/loading architecture


In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)


In [ ]:
history = model.fit(
    training_dataset,
    epochs = 100,
    validation_data = test_dataset,
    batch_size = 32,
    validation_steps = int(0.25*len(test_dataset)),
    callbacks = [stop_on_accuracy,early_stopping,tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)]
)
model.save("/content/Drive/MyDrive/finalfinalfinalp2.h5")

In [ ]:
model.save("/content/Drive/MyDrive/finalfinalfinalp2.h5")

In [ ]:
import matplotlib.pyplot as plt

train_acc = history.history["accuracy"]
test_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs_range = range(1,44)

plt.figure(figsize=(18,5))
plt.subplot(1,2,1)
plt.plot(epochs_range,train_acc,label="Training Accuracy",marker="o")
plt.plot(epochs_range,test_acc,label="Validation Accuracy",marker="o")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Training v/s Validation Accuracy")
plt.legend()
plt.grid()

plt.figure(figsize=(18,5))
plt.subplot(1,2,2)
plt.plot(epochs_range,loss,label = "Training_loss",marker="o")
plt.plot(epochs_range,val_loss, label="Validation_loss",marker="o")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training v/s Validation loss")
plt.legend()
plt.grid()

plt.show()



In [ ]:
model_x = tf.keras.models.load_model("/content/Drive/MyDrive/finalfinalfinalp2.h5")

In [ ]:
model_x.evaluate(test_dataset)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Class labels
class_names = ['glioma', 'meningioma', 'notumor', 'pituitary']

def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

    # Optional: print classification report
    print("\nClassification Report:\n")
    print(classification_report(y_true, y_pred, target_names=class_names))


plot_confusion_matrix(test_dataset.classes, np.argmax(model_x.predict(test_dataset), axis=-1))